### 28.04.25, © Dmytro Sokhin KI-21-1, 2025

Source: https://github.com/https-deeplearning-ai/tensorflow-1-public

# Workshop 1 Assignment 2: Implementing Callbacks in TensorFlow using the MNIST Dataset

Write an MNIST classifier that trains to 99% accuracy and stops once this threshold is achieved. In the lecture you saw how this was done for the loss but here you will be using accuracy instead.

Some notes:
1. Your network should succeed in less than 9 epochs.
2. When it reaches 99% or greater it should print out the string "Reached 99% accuracy so cancelling training!" and stop training.
3. If you add any additional variables, make sure you use the same names as the ones used in the class. This is important for the function signatures (the parameters and names) of the callbacks.

In [ ]:
import os
import tensorflow as tf
from tensorflow import keras

## Load and inspect the data

Begin by loading the data. A couple of things to notice:

- The file `mnist.npz` is already included in the current workspace under the `data` directory. By default the `load_data` from Keras accepts a path relative to `~/.keras/datasets` but in this case it is stored somewhere else, as a result of this, you need to specify the full path.

- `load_data` returns the train and test sets in the form of the tuples `(x_train, y_train), (x_test, y_test)` but in this exercise you will be needing only the train set so you can ignore the second tuple.

In [13]:
current_dir = os.getcwd()
print(f"Current Working Directory: {current_dir}")

data_dir = os.path.join(current_dir, "Workshop", "data") 
data_path = os.path.join(data_dir, "mnist.npz") 
print(f"Attempting to load data from: {data_path}") 

if os.path.exists(data_path):
    with np.load(data_path) as data:
        x_train = data['x_train']
        y_train = data['y_train']
    print("Data loaded successfully using np.load().")
    print("x_train shape:", x_train.shape)
    print("y_train shape:", y_train.shape)

else:
    print(f"Error: File not found at {data_path}")
    x_train, y_train = None, None 

if x_train is not None:
    x_train = x_train / 255.0
    print("Data normalization complete.")
else:
    print("Skipping normalization because data was not loaded.")

Current Working Directory: c:\Users\user\Downloads
Attempting to load data from: c:\Users\user\Downloads\Workshop\data\mnist.npz
Data loaded successfully using np.load().
x_train shape: (60000, 28, 28)
y_train shape: (60000,)
Data normalization complete.


Now take a look at the shape of the training data:

In [14]:
data_shape = x_train.shape

print(f"There are {data_shape[0]} examples with shape ({data_shape[1]}, {data_shape[2]})")

There are 60000 examples with shape (28, 28)


## Defining your callback

Now it is time to create your own custom callback. For this complete the `myCallback` class and the `on_epoch_end` method in the cell below. If you need some guidance on how to proceed, check out this [link](https://www.tensorflow.org/guide/keras/custom_callback).

In [ ]:
# GRADED CLASS: myCallback
### START CODE HERE

# Remember to inherit from the correct class -> tf.keras.callbacks.Callback
class myCallback(tf.keras.callbacks.Callback):
        # Define the correct function signature for on_epoch_end
        # Вона приймає 'self', 'epoch' (номер епохи), і 'logs' (словник з метриками)
        def on_epoch_end(self, epoch, logs=None):
            # Перевіряємо, чи існує ключ 'accuracy' у словнику logs
            # та чи його значення більше 0.99
            if logs.get('accuracy') is not None and logs.get('accuracy') > 0.99:
                print("\nReached 99% accuracy so cancelling training!")

                # Stop training once the above condition is met
                # Встановлюємо атрибут моделі stop_training в True
                self.model.stop_training = True

### END CODE HERE

callbacks = myCallback()

## Create and train your model

Now that you have defined your callback it is time to complete the `train_mnist` function below. 

**You must set your model to train for 10 epochs and the callback should fire before the 9th epoch for you to pass this assignment.**

**Hint:**
- Feel free to try the architecture for the neural network that you see fit but in case you need extra help you can check out an architecture that works pretty well at the end of this notebook.

In [ ]:
# GRADED FUNCTION: train_mnist
def train_mnist(x_train, y_train):

    ### START CODE HERE

    # Instantiate the callback class
    callbacks = myCallback() # Створюємо екземпляр нашого колбека

    # Define the model
    model = tf.keras.models.Sequential([
        # Вхідний шар: випрямляє зображення 28x28 в 1D вектор (784 елементи)
        tf.keras.layers.Flatten(input_shape=(28, 28)),

        # Прихований шар: повнозв'язний шар з 512 нейронами та ReLU активацією
        tf.keras.layers.Dense(512, activation=tf.nn.relu),

        # Вихідний шар: повнозв'язний шар з 10 нейронами (по одному для кожного класу 0-9)
        # та softmax активацією для отримання ймовірностей класів
        tf.keras.layers.Dense(10, activation=tf.nn.softmax)
    ])

    # Compile the model
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy', # Використовуємо, коли мітки є цілими числами
                  metrics=['accuracy'])

    # Fit the model for 10 epochs adding the callbacks
    # and save the training history
    history = model.fit(x_train, y_train, epochs=10, callbacks=[callbacks])

    ### END CODE HERE

    return history

Call the `train_mnist` passing in the appropiate parameters to get the training history:

In [17]:
hist = train_mnist(x_train, y_train)

c:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 20s 10ms/step - accuracy: 0.8965 - loss: 0.3521
Epoch 2/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 20s 9ms/step - accuracy: 0.9750 - loss: 0.0831
Epoch 3/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 19s 10ms/step - accuracy: 0.9843 - loss: 0.0523
Epoch 4/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 22s 11ms/step - accuracy: 0.9889 - loss: 0.0346
Epoch 5/10
1870/1875 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9919 - loss: 0.0252
Reached 99% accuracy so cancelling training!
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 19s 10ms/step - accuracy: 0.9919 - loss: 0.0253


If you see the message `Reached 99% accuracy so cancelling training!` printed out after less than 9 epochs it means your callback worked as expected. 